In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [6]:
import imutils
import os, shutil, glob
from zipfile import ZipFile
import numpy as np
import matplotlib.pyplot as plt
import PIL
from PIL import UnidentifiedImageError

import cv2
from google.colab.patches import cv2_imshow

# Automation collect data

In [3]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename, quality):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      //await new Promise((resolve) => capture.onclick = resolve);
      await new Promise(r => setTimeout(r, 1500));

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [4]:
data_path = './dataset'
crop_path = './crop'
try: 
  os.mkdir(data_path)
except FileExistsError:
  print(f'Folder\'s {data_path} Existed ')

try:
  os.mkdir(crop_path)
except FileExistsError:
  print(f'Folder\'s {crop_path} Existed ')

In [ ]:
from IPython.display import Image

n_image = 1
session = ['front','front_noClasses', 'left', 'left_noClasses', 'right', 'right_noClasses']

for i in range(n_image):
  try:
    filename = take_photo(filename= data_path + f'/tuanle_{session[3]}_{50+5}.jpg', quality=0.8)

    print('Saved to {}'.format(filename))
    
    # Show the image which was just taken.
    display(Image(filename))
  except Exception as err:
    # Errors will be thrown if the user does not have a webcam or if they do not
    # grant the page permission to access it.
    print(str(err))

In [14]:
# shutil.copy(save_path + '/dataset_600.zip', '/content/dataset')
with ZipFile(f'./dataset/dataset_600.zip','r') as zip:
  zip.extractall()
  print('All files unzipped successfully!')  

All files unzipped successfully!


In [33]:
def zip_and_save(file_zip_name, file_paths, save_path): 
  # writing files to a zipfile
  with ZipFile(f'./{file_zip_name}.zip','w') as zip:
    # writing each file one by one
    for file in file_paths:
      zip.write(file)
    
    print('All files zipped successfully!')  

  # Copy dataset.zip to my own drive
  shutil.copy(f'./{file_zip_name}.zip', save_path)


# Double check the dataset folder
# Count the file in dataset folder
dataset_list = sorted(glob.glob(data_path + '/*'))
print(f'The number of image: {len(dataset_list)}')
print(f'{dataset_list[:5]} \n')

save_path = '/content/drive/MyDrive/Colab_Notebooks/Week_9/dataset'
save_path2 = '/content/drive/MyDrive/Auto_Data_Collection/faces/TuanLe'
zip_and_save('dataset_600', dataset_list, save_path)
zip_and_save('dataset_600', dataset_list, save_path2)


The number of image: 600
['./dataset/tuanle_front_0.jpg', './dataset/tuanle_front_1.jpg', './dataset/tuanle_front_10.jpg', './dataset/tuanle_front_11.jpg', './dataset/tuanle_front_12.jpg'] 

All files zipped successfully!
All files zipped successfully!


# Facial_Detect_image

Load and use the pre-trained YOLO model

In [17]:
MODEL = '/content/drive/MyDrive/Colab_Notebooks/Week_9/yolov3-face.cfg'
WEIGHT = '/content/drive/MyDrive/Colab_Notebooks/Week_9/yolov3-wider_16000.weights'

net = cv2.dnn.readNetFromDarknet(MODEL, WEIGHT)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

### Drop image

In [27]:
# drop image and save in folder
IMG_WIDTH, IMG_HEIGHT = 416, 416

def drop_image(images_path):
  frame = cv2.imread(images_path)
  print(images_path.split('/')[-1])
  # cv2_imshow(frame)
  blob = cv2.dnn.blobFromImage(frame, 1/255, (IMG_WIDTH, IMG_HEIGHT), [0, 0, 0], 1, crop=False)
  ##############################################################################################
  net.setInput(blob)
  output_layers = net.getUnconnectedOutLayersNames()
  outs = net.forward(output_layers)
  
  frame_height = frame.shape[0]
  frame_width = frame.shape[1]
  #####################################################################################################
  confidences = []
  boxes = []
  
  for out in outs:
    for detection in out:
      confidence = detection[-1] 
      if confidence > 0.5: 
        center_x = int(detection[0] * frame_width) 
        center_y = int(detection[1] * frame_height)
        width = int(detection[2] * frame_width)
        height = int(detection[3] * frame_height)
        
        topleft_x = center_x - (width/2)
        topleft_y = center_y - (height/2)
        confidences.append(float(confidence))
        boxes.append([topleft_x, topleft_y, width, height])
        
  indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
  ##############################################################################################
  final_boxes = []
  result = frame.copy()
  for i in indices:
    i = i[0]
    box = boxes[i]
    final_boxes.append(box)
    # print(len(final_boxes)) # for debugging

    extend_dim = 5
    left = int(box[0]) - extend_dim
    top = int(box[1]) - extend_dim
    width = int(box[2]) + extend_dim
    height = int(box[3]) + extend_dim
    # cv2_imshow(frame[top:top+height, left: left+width])
        
    return frame[top:top+height, left: left+width]


In [96]:
dataset_list[148]

'./dataset/tuanle_front_noClasses_52.jpg'

In [34]:
for image in range(350,len(dataset_list)):
  print(image)
  crop_image = drop_image(dataset_list[image])
  a = dataset_list[image].split('/')[-1]
  cv2.imwrite(crop_path + f'/crop_{a}', crop_image)

350
tuanle_left_noClasses_54.jpg
351
tuanle_left_noClasses_55.jpg
352
tuanle_left_noClasses_56.jpg
353
tuanle_left_noClasses_57.jpg
354
tuanle_left_noClasses_58.jpg
355
tuanle_left_noClasses_59.jpg
356
tuanle_left_noClasses_6.jpg
357
tuanle_left_noClasses_60.jpg
358
tuanle_left_noClasses_61.jpg
359
tuanle_left_noClasses_62.jpg
360
tuanle_left_noClasses_63.jpg
361
tuanle_left_noClasses_64.jpg
362
tuanle_left_noClasses_65.jpg
363
tuanle_left_noClasses_66.jpg
364
tuanle_left_noClasses_67.jpg
365
tuanle_left_noClasses_68.jpg
366
tuanle_left_noClasses_69.jpg
367
tuanle_left_noClasses_7.jpg
368
tuanle_left_noClasses_70.jpg
369
tuanle_left_noClasses_71.jpg
370
tuanle_left_noClasses_72.jpg
371
tuanle_left_noClasses_73.jpg
372
tuanle_left_noClasses_74.jpg
373
tuanle_left_noClasses_75.jpg
374
tuanle_left_noClasses_76.jpg
375
tuanle_left_noClasses_77.jpg
376
tuanle_left_noClasses_78.jpg
377
tuanle_left_noClasses_79.jpg
378
tuanle_left_noClasses_8.jpg
379
tuanle_left_noClasses_80.jpg
380
tuanle_le

In [39]:
dataset_crop_list = glob.glob(crop_path + '/*.jpg')
print(f'The number of image: {len(dataset_crop_list)}')
zip_and_save('crop_600', dataset_crop_list, save_path2)
zip_and_save('crop_600', dataset_crop_list, save_path)

The number of image: 600
All files zipped successfully!
All files zipped successfully!
